In [17]:
import sys
from time import sleep
import pygame
from ipynb.fs.full.bullet import Bullet
from ipynb.fs.full.settings import Settings
from ipynb.fs.full.alien import Alien

#watch for keyboard and mouse events 

def check_events(game_settings,screen, ship, bullets):
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            sys.exit()     
            
        elif event.type == pygame.KEYDOWN:
            check_keydown_events(event, game_settings, screen, ship, bullets)
                          
        elif event.type == pygame.KEYUP:
            check_keyup_events(event,ship)

def check_keydown_events(event, game_settings, screen, ship, bullets):
    if event.key == pygame.K_RIGHT:
        ship.moving_right = True
        
    elif event.key == pygame.K_LEFT:
        ship.moving_left = True    
    
    elif event.key == pygame.K_SPACE:
        fire_bullet(game_settings,screen,ship,bullets)
    
    elif event.key == pygame.K_q:
        sys.exit()
        
def check_keyup_events(event,ship):
    if event.key == pygame.K_RIGHT:
        ship.moving_right = False
        
    elif event.key == pygame.K_LEFT:
        ship.moving_left = False

def fire_bullet(game_settings,screen,ship,bullets):
    #create a new bullet and add to the bullets group
    if len(bullets) < game_settings.bullets_allowed:
        new_bullet = Bullet(game_settings, screen, ship)
        bullets.add(new_bullet)

def update_bullets(game_settings, screen, ship, aliens, bullets):
    bullets.update()
    
    #get rid of bullets that disappear
    for bullet in bullets.copy():
        if bullet.rect.bottom<=0:
            bullets.remove(bullet)   
    
    check_bullet_alien_collisions(game_settings, screen, ship, aliens, bullets)
    
def check_bullet_alien_collisions(game_settings, screen, ship, aliens, bullets):
    #check whether any bullets have hit aliens, if so get rid of alien -true and bullet -true
    collisions = pygame.sprite.groupcollide(bullets,aliens, True,True)

    if len(aliens) == 0:
        #destroy existing bullets and create new fleet
        bullets.empty()
        create_fleet(game_settings, screen, ship, aliens)
        
def get_number_aliens_x(game_settings, alien_width):
    #find out how many aliens fit in a row
    available_space_x = game_settings.screen_width - 2 * alien_width
    number_aliens_x = int(available_space_x / (2*alien_width))
    return number_aliens_x

def get_number_rows(game_settings, ship_height, alien_height):
    available_space_y = (game_settings.screen_height - (2*alien_height) - ship_height)
    number_rows = int(available_space_y / (2*alien_height))
    return number_rows

def create_alien(game_settings,screen,aliens, alien_number,row_number):
    alien = Alien(game_settings,screen)
    alien_width = alien.rect.width
    alien_height = alien.rect.height
    alien.x = alien_width + 2* alien_width * alien_number
    alien.rect.x = alien.x
    alien.rect.y = alien.rect.height + 2*alien.rect.height * row_number
    aliens.add(alien)
    
def create_fleet(game_settings, screen,ship, aliens):
#create the first fleet of aliens in a row
    
    alien = Alien(game_settings,screen)
    number_aliens_x = get_number_aliens_x(game_settings, alien.rect.width)
    number_rows = get_number_rows(game_settings, ship.rect.height,alien.rect.height)
     
    #create the fleet
    for row_number in range(number_rows):
        for alien_number in range(number_aliens_x):
            create_alien(game_settings, screen, aliens, alien_number, row_number)

             

def check_fleet_edges(game_settings, aliens):
    #respond if aliens touch the edge
    for alien in aliens.sprites():
        if alien.check_edges():
            change_fleet_direction(game_settings, aliens)
            break
            
def change_fleet_direction(game_settings,aliens):
    #drop fleet and change direction
    for alien in aliens.sprites():
        alien.rect.y += game_settings.fleet_drop_speed
    game_settings.fleet_direction *= -1
        


        
def ship_hit(game_settings,stats,screen,ship,aliens,bullets):
    #decrease ships_left
    if stats.ships_left > 0:
        stats.ships_left -=1
    
        aliens.empty()
        bullets.empty()
    
        create_fleet(game_settings,screen,ship,aliens)
        ship.center_ship()
    
        sleep(0.5)
    else:
        stats.game_active = False

def check_aliens_bottom(game_settings,stats,screen,ship,aliens,bullets):
    #check if any aliens reach bottom
    screen_rect = screen.get_rect()
    for alien in aliens.sprites():
        if alien.rect.bottom >= screen_rect.bottom:
            #treat this the same as if the ship got hit
            ship_hit(game_settings,stats,screen,ship,aliens,bullets)
            break
            
def update_aliens(game_settings, stats, screen, ship, aliens, bullets):
    #check if near edge
    check_fleet_edges(game_settings, aliens)
    aliens.update()
    #check if alien-ship collide
    if pygame.sprite.spritecollideany(ship,aliens):
        ship_hit(game_settings, stats, screen, ship, aliens, bullets) 
    #look for aliens hitting bottom of the screen
    check_aliens_bottom(game_settings,stats,screen,ship,aliens,bullets)
#redraw the screen during each pass through the loop 

def update_screen(game_settings, screen, ship, aliens, bullets):
    screen.fill(game_settings.bg_color)
    
    #redraw all bullets behind ship and aliens
    for bullet in bullets.sprites():
        bullet.draw_bullet()
        
    ship.blitme()
    aliens.draw(screen)
        
    #make most recent display show
    pygame.display.flip()
